Установим датасет и проверим, соответствует ли он требованиям длины:

In [14]:
from datasets import load_dataset

dataset = load_dataset("inkoziev/incomplete_utterance_restoration")
df = dataset["train"].to_pandas()


text_column = df.select_dtypes(include=['object']).columns[0]

text = df[text_column]
total_chars = text.str.len().sum()
total_words = text.str.split().str.len().sum()

print(f"Общий объём текста: {total_chars} символов, {total_words} слов.")
print(text.head(3))

Общий объём текста: 3493493 символов, 586661 слов.
0    я учусь в восьмом классе
1    я считаю себя изучателем
2              План не сложен
Name: expanded_phrase, dtype: object


Данный датасет имеет достаточно слов для дальнейшего обучения НС. Продолжим работу с датасетом - разделим предложения на слова и создадим словарь слов и их индексов для быстрого поиска.



In [15]:
import random

vocab = set()

L = 6
sentences = [sentence.split() for sentence in text]
print(sentences[:5])

# поделим предложения на уникальные слова
for sentence in sentences:
    vocab.update(sentence)

vocab = list(vocab)
print(vocab[:10], len(vocab))

word_indx = {w: i for i, w in enumerate(vocab)}
print(word_indx["я"])

[['я', 'учусь', 'в', 'восьмом', 'классе'], ['я', 'считаю', 'себя', 'изучателем'], ['План', 'не', 'сложен'], ['я', 'неплохо', 'поживаю'], ['Смертен', 'ли', 'Гиппарх?']]
['музыкальный', '1964', 'конструированием', 'присущ', 'дверь,', '100*86=8600', 'Литератор', 'радоваться,', 'счетчика', 'бабелевские'] 71663
18614


Создадим положительные (реальные пары) и отрицательные примеры (случайные пары):

In [16]:
positiv_pairs = []
for sentence in sentences:
    for i in range(len(sentence)):
        target = sentence[i]
        for j in range(max(0, i - L), min(i + L + 1, len(sentence))):
            if i != j:
                positiv_pairs.append((target, sentence[j]))

print(positiv_pairs[:15])

[('я', 'учусь'), ('я', 'в'), ('я', 'восьмом'), ('я', 'классе'), ('учусь', 'я'), ('учусь', 'в'), ('учусь', 'восьмом'), ('учусь', 'классе'), ('в', 'я'), ('в', 'учусь'), ('в', 'восьмом'), ('в', 'классе'), ('восьмом', 'я'), ('восьмом', 'учусь'), ('восьмом', 'в')]


In [17]:
negative_pairs = []
for target, context in positiv_pairs:
    neg_context = random.choice(vocab)
    while neg_context == context:
        neg_context = random.choice(vocab)
    negative_pairs.append((target, neg_context))

print(negative_pairs[:15])

[('я', 'верующих'), ('я', 'Морковь'), ('я', 'поэбатле'), ('я', 'боженька,'), ('учусь', 'рукав'), ('учусь', '3-4'), ('учусь', 'тренировке'), ('учусь', 'Карамазовы",'), ('в', 'килограммов'), ('в', 'крематорию'), ('в', 'сакральный'), ('в', 'эстафеты'), ('восьмом', 'радости?'), ('восьмом', 'честь.'), ('восьмом', 'черепахах')]


Создадим сбалансированный датасет, состоящий как из положительных, так и из отрицательных примеров:

In [18]:
import numpy as np

X = [] # пары (индексы пар)
Y = [] # метки (для реальных пар 1, для случайных 0)

for target, contex in positiv_pairs:
    X.append([word_indx[target], word_indx[contex]])
    Y.append(1)

for target, neg_context in negative_pairs:
    X.append([word_indx[target], word_indx[neg_context]])

X = np.array(X)
Y = np.array(Y)
print(X[:5], Y[:5])


[[18614  9118]
 [18614  5128]
 [18614 38686]
 [18614 51794]
 [ 9118 18614]] [1 1 1 1 1]


Создадим класс Word2Vec для реализации эмбедингов и обучения и создадим и обучим полносвязную нейронную сеть с одним слоем (dh = 500):

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow.keras.models

from sklearn.metrics.pairwise import cosine_similarity

def show_similar_words(emb_matrix, word_indx, vocab, word, top_n=10):
    if word not in word_indx:
        print(f"Слово '{word}' не найдено в словаре.")
        return
    vec = emb_matrix[word_indx[word]].reshape(1, -1)
    similarities = cosine_similarity(vec, emb_matrix)[0]
    
    # отсортируем и выведем top_n похожих слов
    similar_ids = np.argsort(-similarities)[:top_n + 1]
    print(f"Слова, похожие на '{word}':")
    for idx in similar_ids:
        if vocab[idx] != word:
            print(f"{vocab[idx]} (сходство: {similarities[idx]:.3f})")

class Word2Vec:
    def __init__(self, vocab_size, embedding_dim, lambda_coef=0.001):
        self.W = np.random.randn(vocab_size, embedding_dim) * lambda_coef
        self.C = np.random.randn(vocab_size, embedding_dim) * lambda_coef
    
    def sigmoid(self, x):
        x = np.clip(x, -50, 50)
        return 1 / (1 + np.exp(-x))
    
    def train(self, positiv_pairs, negative_pairs, learning_rate=0.0005, epochs=7):
        for epoch in range(epochs):
            total_loss = 0

            for target, context in positiv_pairs:
                target_ind = word_indx[target]
                context_ind = word_indx[context]

                score = np.dot(self.W[target_ind], self.C[context_ind]) 
                prob = max(self.sigmoid(score), 1e-10)
                loss = -np.log(prob)
                total_loss += loss

                grad = (1 - prob) * learning_rate
                self.W[target_ind] += grad  * self.C[context_ind]
                self.C[context_ind] += grad * self.W[target_ind]

            for target, neg_context in negative_pairs:
                target_ind = word_indx[target]
                neg_ind = word_indx[neg_context]

                score = np.dot(self.W[target_ind], self.C[neg_ind])
                prob = self.sigmoid(-score)
                loss = -np.log(prob)
                total_loss += loss

                grad = -prob * learning_rate
                self.W[target_ind] += grad * self.C[neg_ind]
                self.C[neg_ind] += grad * self.W[target_ind]
            
            print(f"Эпоха: {epoch}, ошибка: {total_loss/len(positiv_pairs + negative_pairs):.4f}")
        
        return self.W

# подготовка данных для нейросети
def prepare_data(sentences, L):
    X, y = [], []
    for sentence in sentences:
        for i in range(L, len(sentence)):
            context = sentence[i-L:i]
            target = sentence[i]
            if all(word in word_indx for word in context) and target in word_indx:
                X.append([word_indx[word] for word in context])
                y.append(word_indx[target])
    return np.array(X), np.array(y)

dh = 500
embedding_d = [100, 500, 1000]
models = {}
histories = {}

for d in embedding_d:
    print(f"Обучение модели с длиной эмбединга {d}")
    
    print("Обучение W2V")
    w2v = Word2Vec(len(vocab), d)
    emb_matrix = w2v.train(positiv_pairs, negative_pairs)
    
    show_similar_words(emb_matrix, word_indx, vocab, "король")

    X, y = prepare_data(sentences, L)
    print("Создание и обучение нейросети")

    model = Sequential([
        Embedding(len(vocab), d, weights=[emb_matrix], input_length=L, trainable=True),
        Flatten(),
        Dense(dh, activation='relu'),
        Dense(len(vocab), activation='softmax')
    ])

    model.compile(optimizer=Adam(), 
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    early_stopping = EarlyStopping(
        monitor='val_accuracy', 
        patience=2, 
        mode='max'
    )

    history = model.fit(
        X, y, 
        epochs=15,
        shuffle=True,
        batch_size=512,
        validation_split=0.2,
        callbacks=[early_stopping]
    )
    
    models[d] = model

    histories[d] = history


Обучение модели с длиной эмбединга 100
Обучение W2V
Эпоха: 0, ошибка: 0.6930
Эпоха: 1, ошибка: 0.6743
Эпоха: 2, ошибка: 0.6341
Эпоха: 3, ошибка: 0.6085
Эпоха: 4, ошибка: 0.5892
Эпоха: 5, ошибка: 0.5720
Эпоха: 6, ошибка: 0.5553
Слова, похожие на 'король':
посещение (сходство: 0.986)
кормлю (сходство: 0.986)
родился (сходство: 0.986)
найдешь (сходство: 0.985)
должны (сходство: 0.985)
потерял (сходство: 0.985)
закажу (сходство: 0.985)
проявляется, (сходство: 0.985)
живой (сходство: 0.985)
отдыхать (сходство: 0.985)
Создание и обучение нейросети
Epoch 1/15


/home/ksenia/.local/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


60/60 ━━━━━━━━━━━━━━━━━━━━ 85s 1s/step - accuracy: 0.0213 - loss: 10.1051 - val_accuracy: 0.0538 - val_loss: 8.6541
Epoch 2/15
60/60 ━━━━━━━━━━━━━━━━━━━━ 82s 1s/step - accuracy: 0.0652 - loss: 7.3442 - val_accuracy: 0.0798 - val_loss: 8.6346
Epoch 3/15
60/60 ━━━━━━━━━━━━━━━━━━━━ 83s 1s/step - accuracy: 0.0860 - loss: 6.3433 - val_accuracy: 0.1016 - val_loss: 9.1119
Epoch 4/15
60/60 ━━━━━━━━━━━━━━━━━━━━ 83s 1s/step - accuracy: 0.1082 - loss: 5.6410 - val_accuracy: 0.1234 - val_loss: 9.7235
Epoch 5/15
60/60 ━━━━━━━━━━━━━━━━━━━━ 83s 1s/step - accuracy: 0.1308 - loss: 5.0670 - val_accuracy: 0.1432 - val_loss: 10.3966
Epoch 6/15
60/60 ━━━━━━━━━━━━━━━━━━━━ 83s 1s/step - accuracy: 0.1702 - loss: 4.4959 - val_accuracy: 0.1569 - val_loss: 10.9319
Epoch 7/15
60/60 ━━━━━━━━━━━━━━━━━━━━ 83s 1s/step - accuracy: 0.2297 - loss: 3.9671 - val_accuracy: 0.1714 - val_loss: 11.5625
Epoch 8/15
60/60 ━━━━━━━━━━━━━━━━━━━━ 83s 1s/step - accuracy: 0.2991 - loss: 3.4392 - val_accuracy: 0.1880 - val_loss: 12.271

ValueError: Invalid filepath extension for saving. Please add either a `.keras` extension for the native Keras format (recommended) or a `.h5` extension. Use `model.export(filepath)` if you want to export a SavedModel for use with TFLite/TFServing/etc. Received: filepath=model_100.

Протестируем нейронную сеть на примере:

In [ ]:

def predict_next_word(models, vocab, word_indx, test_sentence, L, embedding_d):
    # токенизация и подготовка контекста
    test_words = test_sentence.split()
    
    # проверка, что контекст достаточной длины
    if len(test_words) < L:
        print(f"Ошибка: предложение слишком короткое. Нужно минимум {L} слов, а получено {len(test_words)}")
        return
    
    # Бебрём последние L слов как контекст
    context = test_words[-L:]
    print(f"\nТестовый контекст: {' '.join(context)}")
    
    # преобразуем слова в индексы с обработкой неизвестных слов
    test_indices = []
    unk_token = "<UNK>"
    
    for word in context:
        if word in word_indx:
            test_indices.append(word_indx[word])
        else:
            print(f"Слово '{word}' не найдено в словаре. Заменяю на {unk_token}")
            if unk_token in word_indx:
                test_indices.append(word_indx[unk_token])
            else:
                print(f"Ошибка: токен {unk_token} отсутствует в словаре")
                return
    
    # предсказание для каждой модели
    for d in embedding_d:
        if d not in models:
            print(f"Модель с d={d} не найдена")
            continue
            
        print(f"\nПредсказание для модели с d={d}:")
        test_X = np.array([test_indices])
        
        try:
            pred = models[d].predict(test_X, verbose=0)
            predicted_idx = np.argmax(pred[0])
            predicted_word = vocab[predicted_idx]
            print(f"Следующее слово: '{predicted_word}' (вероятность: {np.max(pred[0]):.2%})")
            
            # Топ-5 вариантов
            top_indices = np.argsort(-pred[0])[:5]
            print("Топ-5 вариантов:")
            for i, idx in enumerate(top_indices):
                print(f"{i+1}. {vocab[idx]} ({pred[0][idx]:.2%})")
                
        except Exception as e:
            print(f"Ошибка предсказания: {str(e)}")

test_sentence = "я учусь в восьмом классе я считаю себя"
predict_next_word(
    {100: tensorflow.keras.models.load_model("model_100"), 500: tensorflow.keras.models.load_model("model_500"), 1000: tensorflow.keras.models.load_model("model_1000")},
    vocab, word_indx, test_sentence, L, [100, 500, 1000])


NameError: name 'models' is not defined